In [8]:
# read data 
import pandas as pd


df = pd.read_csv('train.csv')
df.head()

,label,content,synonym_aug,back_translation
0,3,lớp này không phải wzjwz124 hướng dẫn thực hành .,vỉa này không phải wzjwz124 hướng_dẫn thực_hành .,Lớp học này không phải là ezzzz được thực tế h...
1,3,cần nhiều hoạt động ngoại khóa .,cần nhiều hoạt_động ngoại_khóa .,Rất nhiều hoạt động ngoại khóa.
2,1,kiến thức còn kỳ kỳ .,kiến_thức còn kỳ_kỳ .,Có thời gian cho tri thức.
3,3,không có gì để phàn nàn .,không có gì để phàn_nàn .,Không có gì phải phàn nàn cả.
4,1,cần xem xét lại khi đặt ra những yêu cầu về ch...,cần xem_xét lại khi đặt ra những yêu_cầu về tạ...,"để xem xét các quy định cho các nhóm đối xứng,..."


In [9]:
import random
import json
import re

random.seed(1)

# Load stop words
stop_words = []
with open("../../text_augmentation_vietnamese/vietnamese-stopwords.txt", "r") as f:
    for line in f:
        stop_words.append(line.strip('\n'))

def get_only_chars(line):
    return line

def normalize(text):
    return text.strip().replace("_", " ").lower()

def get_synonyms(word):
    synonyms = set()
    with open("../../text_augmentation_vietnamese/word_net_vi.json", "r") as f:
        wordnet = json.load(f)
    for key, value in wordnet.items():
        if normalize(key) == normalize(word):
            for v in value:
                synonyms.add(normalize(v))
    if normalize(word) in synonyms:
        synonyms.remove(normalize(word))
    return list(synonyms)

def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')
    return new_words

def random_deletion(words, p):
    if len(words) == 1:
        return words
    new_words = []
    for word in words:
        if random.uniform(0, 1) > p:
            new_words.append(word)
    if len(new_words) == 0:
        return [words[random.randint(0, len(words) - 1)]]
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        if len(new_words) > 0:
            new_words = swap_word(new_words)
    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1 and len(new_words) > 0:
        random_word = new_words[random.randint(0, len(new_words) - 1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    if len(new_words) > 0 and len(synonyms) > 0:
        random_synonym = synonyms[0]
        random_idx = random.randint(0, len(new_words) - 1)
        new_words.insert(random_idx, random_synonym)

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word != '']
    num_words = len(words)
    augmented_sentences = []
    if len(words) <= 0:
        return augmented_sentences
    num_new_per_technique = int(num_aug / 4) + 1
    n_sr = max(1, int(alpha_sr * num_words))
    n_ri = max(1, int(alpha_ri * num_words))
    n_rs = max(1, int(alpha_rs * num_words))
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))
    augmented_sentences = list(set(augmented_sentences))
    augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
    augmented_sentences.append(sentence)
    return augmented_sentences

def gen_eda(train_orig, output_file, alpha, num_aug=9):
    writer = open(output_file, 'w')
    lines = open(train_orig, 'r').readlines()
    writer.write("free_text" + "," + "label_id" + '\n')
    augm = ""
    for i, line in enumerate(lines):
        try:
            parts = line[:-1].split('|')
            label = parts[1]
            sentence = parts[0]
            aug_sentences = eda(sentence, alpha_sr=alpha, alpha_ri=alpha, alpha_rs=alpha, p_rd=alpha, num_aug=num_aug)
            for aug_sentence in aug_sentences:
                augm = augm + aug_sentence + "," + label + '\n'
        except Exception as e:
            print(e)
            print(parts)
            pass
    writer.write(augm)
    writer.close()
    print(
        "generated augmented sentences with eda for " + train_orig + " to " + output_file + " with num_aug=" + str(
            num_aug))


In [10]:
# (1) Weak Augmentation: Synonym Replacement
import nlpaug.augmenter.word as naw
from tqdm import tqdm
from underthesea import word_tokenize

def synonym_aug_sentence(sentence):
    # Tách từ tiếng Việt
    words = word_tokenize(sentence, format="text").split()
    n_sr = max(1, int(0.1 * len(words)))
    aug_words = synonym_replacement(words, n_sr)
    return ' '.join(aug_words)

df['synonym_aug'] = df['content'].apply(synonym_aug_sentence)
df.to_csv('train.csv', index=False)

In [11]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

# Chuẩn bị mô hình dịch Việt-Anh và Anh-Việt
vi_en_model_name = 'Helsinki-NLP/opus-mt-vi-en'
en_vi_model_name = 'Helsinki-NLP/opus-mt-en-vi'

vi_en_tokenizer = MarianTokenizer.from_pretrained(vi_en_model_name)
vi_en_model = MarianMTModel.from_pretrained(vi_en_model_name)

en_vi_tokenizer = MarianTokenizer.from_pretrained(en_vi_model_name)
en_vi_model = MarianMTModel.from_pretrained(en_vi_model_name)

def translate(text, tokenizer, model):
    batch = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt")
    gen = model.generate(**batch)
    return tokenizer.batch_decode(gen, skip_special_tokens=True)[0]

def back_translate_vi(text):
    en = translate(text, vi_en_tokenizer, vi_en_model)
    vi_bt = translate(en, en_vi_tokenizer, en_vi_model)
    return vi_bt

# Ví dụ áp dụng với pandas DataFrame
tqdm.pandas()
df['back_translation'] = df['content'].progress_apply(back_translate_vi)
df.to_csv('train.csv', index=False)
print("Đã lưu file train.csv với cột back_translation!")


c:\Users\HP\Anaconda3\envs\jointmatch\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  0%|          | 0/400 [00:00<?, ?it/s]c:\Users\HP\Anaconda3\envs\jointmatch\lib\site-packages\transformers\tokenization_utils_base.py:3668: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of 

Đã lưu file train.csv với cột back_translation!


In [12]:
df.head()

,label,content,synonym_aug,back_translation
0,3,lớp này không phải wzjwz124 hướng dẫn thực hành .,vỉa này không phải wzjwz124 hướng_dẫn thực_hành .,Lớp học này không phải là ezzzz được thực tế h...
1,3,cần nhiều hoạt động ngoại khóa .,cần nhiều vui chơi ngoại_khóa .,Rất nhiều hoạt động ngoại khóa.
2,1,kiến thức còn kỳ kỳ .,học thức còn kỳ_kỳ .,Có thời gian cho tri thức.
3,3,không có gì để phàn nàn .,không có gì để phàn_nàn .,Không có gì phải phàn nàn cả.
4,1,cần xem xét lại khi đặt ra những yêu cầu về ch...,cần xét đến lại khi đặt ra những yêu_cầu về tạ...,"để xem xét các quy định cho các nhóm đối xứng,..."


In [13]:
df_train = pd.read_csv('train.csv')

df_train['label'].value_counts()

3    100
1    100
2    100
0    100
Name: label, dtype: int64

In [2]:
import pandas as pd

df = pd.read_csv('dev.csv')  # Thay bằng tên file thực tế của bạn
mapping = {4: 3, 3: 2, 2: 1, 1: 0}
df['label'] = df['label'].map(mapping).fillna(df['label']).astype(int)
df.to_csv('dev.csv', index=False)  # Lưu thành file mới



In [14]:
import pandas as pd

df = pd.read_csv('dev.csv')
target_per_class = 20

sampled_df = pd.concat([
    df[df['label'] == k].sample(n=target_per_class, random_state=42)
    for k in df['label'].unique()
])

sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)
sampled_df.to_csv('dev.csv', index=False, encoding='utf-8')
print("✅ Đã lưu file dev.csv cho mỗi lớp!")


✅ Đã lưu file dev.csv cho mỗi lớp!
